In [ ]:
from sasmodels.core import load_model
from sasmodels.bumps_model import Model, Experiment
from sasdata.dataloader.loader import Loader

from bumps.names import *
from bumps.fitters import fit
from bumps.formatnum import format_uncertainty

import matplotlib.pyplot as plt

In [ ]:
loader = Loader()

In [ ]:
test_data = loader.load('../simulated_sans_data.csv')[0]

#The sasdata requires explicit setup of qmin, qmax and mask
test_data.qmin = test_data.xmin
test_data.qmax = test_data.xmax
test_data.mask = np.isnan(test_data.y)

print(test_data) # Good to see how much input metadata it can handle already 

In [ ]:
#TODO: Print and plot data 
plt.plot(test_data.x, test_data.y)

plt.yscale('log')
plt.xscale('log')

In [ ]:
kernel = load_model('cylinder', platform='cpu')

rad_guess = 5
len_guess = 20

pars = dict(radius=rad_guess, 
            length=len_guess,
            background=0.01,
            scale=0.1,
            sld=2.0,
            sld_solvent=3.0)
model = Model(kernel, **pars)

deviator = 30
bgdev = 0.01

# SET THE FITTING PARAMETERS
model.radius.range(rad_guess/rad_guess, rad_guess*rad_guess)
model.length.range(len_guess/len_guess, len_guess*len_guess)
model.scale.range(0, 1)
model.background.range(0, 1)

In [ ]:
M = Experiment(data=test_data, model=model)
problem = FitProblem(M)

In [ ]:
problem.plot()

In [ ]:
import os
os.environ["HAVE_OPENCL"] = "0"
os.environ['SAS_OPEN_CL'] = "none"
print(f"Initial chisq {problem.chisq()}")
problem.plot()

In [ ]:
result = fit(problem, method='amoeba')
print(f"Final chisq {problem.chisq()}")
for k, v, dv in zip(problem.labels(), result.x, result.dx):
    print(f"{k} : {format_uncertainty(v, dv)}")
problem.plot()